In [1]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

# Annealed importance sampling

Paper: https://arxiv.org/abs/physics/9803008



# AIS implementation

Let's first define some distributions that we want to use for $f_n$ and $f_0$. In this example, I will use cauchy distribution for $f_n$ and Gaussian for $f_0$. Note that we do not need to know the exact PDF for both $f_n$ and $f_0$. For example, I define the $\textrm{gaussian_num}$ as just 

$$e^{-0.5(\frac{x - \mu}{\sigma})^2}$$

because Gaussian PDF is proportional to this quantity.

In [2]:
def cauchy(x, x_0 = 0, gamma = 0.5):
    return (1 / (np.pi * gamma)) * ((gamma ** 2) / ((x - x_0)**2 + gamma**2))

def gaussian_num(x, mu = 0, sigma = 1):
    return np.exp(-0.5 * ((x - mu) / sigma)**2)

In [3]:
def f_n(x):
    return cauchy(x)
    
def f_0(x):
    return gaussian_num(x, mu = 10, sigma = 2)

We then need to define the sequence transition distributions. Usually, we take the interpolation between $f_n$ and $f_0$ as follows (see Eq. 3 in the paper):

$$f_j(x) = f_0(x)^{\beta_j} f_n(x)^{1 - \beta_j},$$

where $1 = \beta_0 > \beta_1 > \dots > \beta_n = 0$

In [4]:
def f_j(x, beta):
    return f_0(x)**beta * f_n(x)**(1 - beta)

Next, we need to define the transition operator. Some options include Metropolis/Metropolis-Hastings, Gibbs sampling, Hamiltonian Monte Carlo, etc. Here, we will use Metropolis sampling by assuming the proposal $q(x'|x) = \mathcal{N}(x,1)$

In [5]:
def metropolis_sampling(x, f, steps = 10):
    for _ in range(steps):
        x_prime = x + np.random.normal() # sample x' from proposal q(x'|x) = N(x,I)
        a = f(x_prime) / f(x) # compute acceptance probability for Metropolis sampling
        if np.random.uniform() < a: # determine if x' is accepted
            x = x_prime
    return x


def gibbs_sampling(x):
    pass


def hamiltonian_mc(x):
    pass

When computing $w$, it is better to do it in $\log$ space for numerical stability. Normally we compute the weight $w^{(i)}$ for the $i$-th sample $x^{(i)}$ as:

$$w^{(i)} = \frac{f_{n-1}(x_{n-1})}{f_{n}(x_{n-1})} \frac{f_{n-2}(x_{n-2})}{f_{n-1}(x_{n-2})} \dots \frac{f_{0}(x_{0})}{f_{1}(x_{0})}$$

In $\log$ space, we have:

$$\log(w^{(i)}) = \Big[\log(f_{n-1}(x_{n-1})) - \log(f_{n}(x_{n-1}))\Big] + \Big[\log(f_{n-2}(x_{n-2})) - \log(f_{n-1}(x_{n-2}))\Big] + \dots + \Big[\log(f_{0}(x_{0})) - \log(f_{1}(x_{0}))\Big]$$

And since we are in the $\log$ space, we start at $\log(w) = 0$. 

In [6]:
num_sample = 100 # number of samples to generate
num_dist = 200  # number of passing/intermediate distributions
betas = np.linspace(0, 1, num_dist)

# Initialize a list to store the samples and their weights
xs = []
ws = []

# AIS
for _ in range(num_sample):
    x = stats.cauchy.rvs(loc=0, scale=0.5) # sample initial point from proposal p_n
    log_w = 0 # set initial log(w) to 0
    
    # Move from f_n -> f_{n-1} -> ... -> f_0
    for i in range(1, num_dist):
        f = lambda x: f_j(x, betas[i])
        x = metropolis_sampling(x, f, steps = 20) # sample x' from f_j with Metropolis sampling
        log_w += np.log(f_j(x, betas[i])) - np.log(f_j(x, betas[i-1])) # compute weight
    
    # Store the sample and its weight
    xs.append(x)
    ws.append(np.exp(log_w))

Now we can compute the expectation (see Eq. 12 in the paper):

$$\bar{a} = \frac{\sum_{i=1}^N w^{(i)} a(x^{(i)})}{\sum_{i=1}^N w^{(i)}}$$

Which in this case should give us a value close to the mean of the target Gaussian.

In [7]:
# Compute expectation of the target distribution
a_bar = np.dot(ws, xs) / np.sum(ws)

print(a_bar)

9.99670948476493


We should also be able to compute $p_0(x = x_{test})$, since $p_0(x) = \frac{f_0}{Z_0}$, and $\frac{Z_0}{Z_n} = \frac{\sum_{i=1}^N w^{(i)}}{N}$. Since we know the full form of $p_n(x)$, then $Z_n = 1$, so $Z_0 = \frac{\sum_{i=1}^N w^{(i)}}{N}$. To check if the estimated $Z_0$ is close to the actual value, we will compare it with $\sigma \sqrt{2\pi}$ since it is the actual normalization constant for $\mathcal{N}(\mu, \sigma)$. And to check if the estimated $p_0(x = x_{test})$ is close to the actual value, we will evaluate $p_0(x = x_{test})$ using $\textrm{stats.norm.pdf(x_test)}$ from scipy.stats.

In [12]:
x_test = 2.6
Z_0 = np.mean(ws)
p_x_test = f_0(x_test) / Z_0

print('Estimated values\n================')
print('Z_0: ', Z_0)
print('p(x = x_test): ', p_x_test, 
      '\nlog p(x = x_test): ', np.log(p_x_test)
     )

print('\nActual values\n================')
print('Actual Z_0: ', 2 * ((2 * np.pi)**0.5))
print('Actual p(x = x_test): ', stats.norm.pdf(x_test, 10, 2), 
      '\nActual log p(x = x_test): ', np.log(stats.norm.pdf(x_test, 10, 2))
     )

Estimated values
Z_0:  5.654357586938124
p(x = x_test):  0.0001883089670748076 
log p(x = x_test):  -8.577426502219899

Actual values
Actual Z_0:  5.0132565492620005
Actual p(x = x_test):  0.00021239013527537572 
Actual log p(x = x_test):  -8.457085713764618
